In [ ]:
import dspy

lm = dspy.OllamaLocal(model="llama3", base_url="http://localhost:11434")

dspy.settings.configure(lm=lm)

In [ ]:
create_post_analysis = dspy.Predict("journal_entry -> therapeutic_advice")

journal_entry = "I think I have been angry about the ways I've been treated in my family system. I need some tools to help myself calm downin the heat of the moment."

response = create_post_analysis(journal_entry=journal_entry)

print(response.therapeutic_advice)